    <EDA process for feature selection>

    * categorical
        1) chisquare-test
        2) OptimalBinning

    * numeric
        1) ANOVA & Kruskall-wallis test
            * 사용하기 전에
                - 실습에서는 독립형 변수가 numeric일 때 & 종속변수가 categorical 일 때, 혹은
                  독립형 변수가 categorical 일 때 & 종속변수가 numeric일 때 사용함
                - 하지만 둘다 numeric일때 사용하지는 않음
                - 때문에 이 부분을 잘 유념하고 사용할 수 있도록 하기
            * ANOVA (3 prerequisite tests)
                * 정규성 검정
                    - qqplot
                    - shapiro-wilks test
                * 등분산성 검정
                    - 정규성 검정 만족해야 함
                    - 하지만 현실 데이터에서는 정규성을 만족하는 경우가 많지 않음
                    - 그러면 box-cox 변환 등을 사용하여 데이터 스케일링 후 다시 검정해보는 경우가 있음
                * 독립성 검정
            * Kruskall-wallis test

        2) OptimalBinning
        3) VIF
        4) Correlation & Heatmap
        5) skew, kurtosis check

    1) categorical variables

In [ ]:
# prerequisites

target_column = ''
list_categorical_columns = []

In [ ]:
# -- chi2square-test --
# classification에 활용
from scipy.stats import chi2_contingency

list_meaningful_column_by_chi2 = []
for column_name in list_categorical_columns:
    statistic, p_value, _, _ = chi2_contingency(pd.crosstab(df[target_column], df[column_name]))
    if p_value < 0.05: # 의미가 없다는 귀무가설 기각시
        list_meaningful_column_by_chi2.append(column_name)
    print('column: {}, statistic: {}, p_value: {}'.format(column_name, statistic, p_value))
print('# of categorical columns before: ', len(list_categorical_columns))
print('# of selected columns after chi2: ', len(list_meaningful_column_by_chi2))

# -- OptimalBinning --
# classification 문제에서 사용 가능
# !pip install -q optbinning
from optbinning import OptimalBinning
import warnings; warnings.filterwarngins('ignore')

iv_df = []

for i in list_categorical_columns:
    variable = i
    x = df[variable].values
    y = df.credit

    optb = OptimalBinning(name=variable,
                          dtype='categorical',
                          solver='cp',
                          )
    optb.fit(x,y)

    binngin_table = optb.binning_table
    v1 = binning_table.build()

    loop_df = pd.DataFrame({'val' : variable,
                            'IV' : [v1.loc['Totals', 'IV']]})
    iv_df.append(loop_df)

iv_df = pd.concat(iv_df).reset_index(drop=True)
iv_df.sort_values(by=['IV'], ascending=False)


    2) numeric variables

In [ ]:
# prerequisites
target_columns = ''
list_numeric_columns = []

In [ ]:
# -- ANOVA --

# 정규성 검정
# 1. qqplot 그려보기
import stats

plt.figure(figsize=(20,10))
x = 1
plt.subplots_adjust(top=.99, bottom=.01, hspace=.2, wspace=.2)
for column_name in list_numeric_columns:
    plt.subplot(2,3,x)
    x = x+1
    stats.probplot(df[column_name], dist=stats.norm, plot=plt)
    plt.title(column_name)
plt.show()

# 2. shapiro-test
# 귀무가설: 해당 데이터는 정규분포이다
# 대립가설: 해당 데이터는 정규분포가 아니다.
for column_name in list_numeric_columns:
    statistic, p_value = stats.shapiro(df[column_name])
    if p_value >= .05:
        print(column_name)
print('end')

# 등분산성 검정

# 독립성 검정

# -- Kruskal-wallis --
# 정규성 가정이 깨질때 사용
from scipy.stats import kruskal
from tqdm.notebook import tqdm

list_meaningful_column_by_kruskal = []
for column_name in list_numeric_columns:
    list_by_value = []
    for value in df[column_name].dropna().unique():
        df_tmp = df[df[column_name] == value][target_column].dropna()
        list_by_value.append(np.array(df_tmp))
    statistic, p_value = kruskal(*list_by_value)
    if p_value < 0.05: # 의미가 없다는 귀무가설 기각시
        list_meaningful_column_by_kruskal.append(column_name)
    print('column: {}, statistic: {}, p_value: {}'.format(column_name, statistic, p_value))
print('# of numeric columns before: ', len(list_numeric_columns))
print('# of selected columns after chi2: ', len(list_meaningful_column_by_kruskal))

# -- OptimalBinnig --
# classification에서 활용
from optbinning import OptimalBinning
import warnings; warnings.filterwarngins('ignore')

iv_df = []

for i in list_numeric_columns:
    variable = i
    x = df[variable].values
    y = df.credit

    optb = OptimalBinning(name=variable,
                          dtype='numerical',
                          solver='cp',
                          max_n_prebins=3 # bin이 많아지면 해석이 어려워짐
                          )
    optb.fit(x,y)

    binngin_table = optb.binning_table
    v1 = binning_table.build()

    loop_df = pd.DataFrame({'val' : variable,
                            'IV' : [v1.loc['Totals', 'IV']]})
    iv_df.append(loop_df)

iv_df = pd.concat(iv_df).reset_index(drop=True)
iv_df.sort_values(by=['IV'], ascending=False)

# -- correlation & heatmap --

# -- VIF --
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df_with_numerical_columns):
    vif = pd.DataFrame()
    vif['VIF_Factor'] = [variance_inflation_factor(df_with_numerical_columns.values, i) for i in range(df_with_numerical_columns.shape[1])]
    vif['Feature'] = df_with_numerical_columns
    return vif

# vif 값을 보고 다중공선성 판단
# ex) > 5, > 10, > 20
# 전체적인 컬럼의 다중 공선성이 높은 경우, 임계값을 높이는 경우도 고려 가능
df_vif = df[list_numeric_columns].copy()